# Jupyter Notebook implementation

First read the [README](README.md) file if you're new.

This is an example of using the code from Jupyter Notebook.

## Contents

- [Classifiction](#classification-modelnet)
- [Part Segmentation](#part-segmentation-shapenet)
- [Semandic Segmentation](#semantic-segmentation-s3dis)

## Code Tree

- `Pointnet_Pointnet2_pytorch/`
  - `data/*`: Default location for data.
  - `data_utils/*`: Data Loader.
  - `models/*`: Model file, `torch.nn.Module` classes. See below for more details.
  - `log/*`: log and output of trained model.
  - `visualize/*`: Visualization code. This is independent of the main code.
  - `test_*.py`: Predeiction codes.
  - `train_*.py`: Training codes.

## Data directory:

This could be named anything and placed anywhere, as long as its path is passed to the argument.

- `data/`: Data directory, create this manually.
  - `modelnet40_normal_resampled/*`: ModelNet40 dataset.
  - `shapenetcore_partanno_segmentation_benchmark_v0_normal/*`: ShapeNet dataset.
  - `Stanford3dDataset_v1.2_Aligned_Version/*`: S3DIS dataset.
  - (`stanford_indoor3d/*`): S3DIS dataset converted by `data_utils/collect_indoor3d_data.py`.

For details, see below sections for each model.

## Models

When running `main()`, the argument `--model` will specify the model to be used.
It will search the directory `models/` for a file name that matches the argument.
Create a new file in the `models/` directory to add a new model.

### Aliases

Core Model

- `pointnet_*.py`: PointNet model
- `pointnet2_*.py`: PointNet++ model

Task Type

- `*_cls*.py`: Classification model
- `*_sem_seg*.py`: Segmentation model
- `*_part_seg*.py`: Part segmentation model

Grouping Method

- `*_msg.py`: Multi-scale grouping model
- `*_ssg.py`: Single-scale grouping model

### Model Directory Tree

- `Pointnet_Pointnet2_pytorch/models/`
  - `pointnet_cls.py`: PointNet classification model
  - `pointnet_part_seg.py`: PointNet part segmentation model
  - `pointnet_sem_seg.py`: PointNet semantic segmentation model
  - `pointnet_utils.py`: PointNet util functions
  - `pointnet2_cls_msg.py`: PointNet++ classification model with multi-scale grouping
  - `pointnet2_cls_ssg.py`: PointNet++ classification model with single-scale grouping
  - `pointnet2_part_seg_msg.py`: PointNet++ part segmentation model with multi-scale grouping
  - `pointnet2_part_seg_ssg.py`: PointNet++ part segmentation model with single-scale grouping
  - `pointnet2_sem_seg_msg.py`: PointNet++ semantic segmentation model with multi-scale grouping
  - `pointnet2_sem_seg_ssg.py`: PointNet++ semantic segmentation model with single-scale grouping
  - `pointnet2_utils.py`: PointNet++ util functions

## Imports

In [1]:
from pprint import pprint
from pathlib import Path

import train_classification
import test_classification

import train_partseg
import test_partseg

import train_semseg
import test_semseg
from data_utils.collect_indoor3d_data import collect_indoor3d_data

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


## **Optional**: External Directory

If you want to put the log and data directories in a different location, you can use the following code.

In [2]:
root_log_dir = Path('..', 'log').resolve()
DATA_DIR = str(Path('..', 'data').resolve())
class_data_dir = str(Path(DATA_DIR, 'modelnet40_normal_resampled').resolve())
partseg_data_dir = str(Path(DATA_DIR, 'shapenetcore_partanno_segmentation_benchmark_v0_normal').resolve())
semseg_raw_data_dir = str(Path(DATA_DIR, 's3dis', 'Stanford3dDataset_v1.2_Aligned_Version').resolve())
semseg_data_dir = str(Path(DATA_DIR, 'stanford_indoor3d').resolve())

root_log_dir, DATA_DIR, class_data_dir, partseg_data_dir, semseg_raw_data_dir, semseg_data_dir

(PosixPath('/home/gokitune/Documents/pointnet-cleanup/log'),
 '/home/gokitune/Documents/pointnet-cleanup/data',
 '/home/gokitune/Documents/pointnet-cleanup/data/modelnet40_normal_resampled',
 '/home/gokitune/Documents/pointnet-cleanup/data/shapenetcore_partanno_segmentation_benchmark_v0_normal',
 '/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version',
 '/home/gokitune/Documents/pointnet-cleanup/data/stanford_indoor3d')

---


## Classification (ModelNet)


### Data

- ModelNet40 dataset in text format
- Download `modelnet40_normal_resampled` from:
  - [Kaggle](https://www.kaggle.com/datasets/chenxaoyu/modelnet-normal-resampled)  
   [Pointcept Huggingface](https://huggingface.co/datasets/Pointcept/modelnet40_normal_resampled-compressed)

Default `--data_dir` is `'data/modelnet40_normal_resampled'`.

- `data/`
  - `modelnet40_normal_resampled/`
    - `train/`
      - `airplane/`
        - `airplane_0001.txt`
        - `airplane_0002.txt`
        - ...
    - ...
    - `file_list.txt`
    - `modelnet10_shape_names.txt`
    - `modelnet10_train.txt`
    - `modelnet10_test.txt`
    - `modelnet40_shape_names.txt`
    - `modelnet40_train.txt`
    - `modelnet40_test.txt`

### **OUTPUT**

- TRAINING: `<log_root>/classification/<args.log_dir or TIME>/checkpoints/best_model.pth`
- PREDICTION: `None (only prints accuracy)`


### Training

`train_classification.py` is used to train the model.

Check all the arguments:

In [3]:
!python train_classification.py -h

usage: training [-h] [--use_cpu] [--gpu GPU] [--batch_size BATCH_SIZE]
                [--model MODEL] [--num_category {10,40}] [--epoch EPOCH]
                [--learning_rate LEARNING_RATE] [--num_point NUM_POINT]
                [--optimizer OPTIMIZER] [--log_root LOG_ROOT]
                [--log_dir LOG_DIR] [--decay_rate DECAY_RATE] [--use_normals]
                [--process_data] [--use_uniform_sample] [--data_dir DATA_DIR]
                [--notebook]

options:
  -h, --help            show this help message and exit
  --use_cpu             use cpu mode
  --gpu GPU             specify gpu device
  --batch_size BATCH_SIZE
                        batch size in training
  --model MODEL         model name [default: pointnet_cls]
  --num_category {10,40}
                        training on ModelNet10/40
  --epoch EPOCH         number of epoch in training
  --learning_rate LEARNING_RATE
                        learning rate in training
  --num_point NUM_POINT
                        Po

The following is the same as running:

```bash
python3 train_classification.py \
     --model pointnet2_cls_ssg \
     --log_dir pointnet2_cls_ssg \
     # --log_root ../log \
     # --data_dir ../data/modelnet40_normal_resampled \
```

In [ ]:
# user defined arguments for command line as a dictionary
args = {
    'model'   : 'pointnet2_cls_ssg',
    'log_dir' : 'pointnet2_cls_ssg',
    'log_root': root_log_dir,
    'data_dir': class_data_dir,
    'notebook': True
}
classification_train_args = train_classification.CommandLineArgs(**args)
train_classification.main(classification_train_args)

### Testing

`test_classification.py` is used to test the model.

Check all the arguments:

In [5]:
!python test_classification.py -h

usage: Testing [-h] [--use_cpu] [--gpu GPU] [--batch_size BATCH_SIZE]
               [--num_category {10,40}] [--num_point NUM_POINT]
               [--log_root LOG_ROOT] --log_dir LOG_DIR [--use_normals]
               [--use_uniform_sample] [--num_votes NUM_VOTES]
               [--data_dir DATA_DIR] [--notebook]

options:
  -h, --help            show this help message and exit
  --use_cpu             use cpu mode
  --gpu GPU             specify gpu device
  --batch_size BATCH_SIZE
                        batch size in training
  --num_category {10,40}
                        training on ModelNet10/40
  --num_point NUM_POINT
                        Point Number
  --log_root LOG_ROOT   Log directory root [default: log]
  --log_dir LOG_DIR     Experiment root within log directory
  --use_normals         use normals
  --use_uniform_sample  use uniform sampiling
  --num_votes NUM_VOTES
                        Aggregate classification scores with voting
  --data_dir DATA_DIR   data direct

The following is the same as running:

```bash
python3 test_classification.py \
     --log_dir pointnet2_cls_ssg \
     # --log_root ../log \
     # --data_dir ../data/modelnet40_normal_resampled \
```

In [6]:
args = {
    'log_dir': 'pointnet2_cls_ssg',
    'log_root': root_log_dir,
    'data_dir': class_data_dir,
    'notebook': True
}
classification_test_args = test_classification.CommandLineArgs(**args)
test_classification.main(classification_test_args)

PARAMETER ...
CommandLineArgs(log_dir='pointnet2_cls_ssg', log_root=PosixPath('/home/gokitune/Documents/pointnet-cleanup/log'), data_dir='/home/gokitune/Documents/pointnet-cleanup/data/modelnet40_normal_resampled', notebook=True, use_cpu=False, gpu='0', batch_size=24, num_category=40, num_point=1024, use_normals=False, use_uniform_sample=False, num_votes=3)
Load dataset ...
The size of test data is 2468


  0%|          | 0/103 [00:00<?, ?it/s]

Test Instance Accuracy: 0.919175, Class Accuracy: 0.882995


---


## Part Segmentation (ShapeNet)

## Data

- ShapeNet dataset in text format
- Download `shapenetcore_partanno_segmentation_benchmark_v0_normal` from:
  - [Kaggle](https://www.kaggle.com/datasets/mitkir/shapenet?resource=download)


Default `--data_dir` is `'data/shapenetcore_partanno_segmentation_benchmark_v0_normal'`.

- `data/`
  - `shapenetcore_partanno_segmentation_benchmark_v0_normal/`
    - `02691156/`
      - `1a04e3eab45ca15dd86060f189eb133.txt`
      - `1a04e3eab45ca15dd86060f189eb133.npz`
      - ...
    - ...
    - `train_test_split/`
      - `shuffled_train_filelist.json`
      - `shuffled_val_filelist.json`
      - `shuffled_test_filelist.json`
    - `utils/`
      - `potential_field.py`
      - `som.py`
    - `synsetoffset2category.txt`

### Data structure

From `data/shapenetcore_partanno_segmentation_benchmark_v0_normal/synsetoffset2category.txt`, the folders correspond to the following categories:

- `Airplane`: 02691156
- `Bag`: 02773838
- `Cap`: 02954340
- `Car`: 02958343
- `Chair`: 03001627
- `Earphone`: 03261776
- `Guitar`: 03467517
- `Knife`: 03624134
- `Lamp`: 03636649
- `Laptop`: 03642806
- `Motorbike`: 03790512
- `Mug`: 03797390
- `Pistol`: 03948459
- `Rocket`: 04099429
- `Skateboard`: 04225987
- `Table`: 04379243

For each .txt file within the folder above, 

 - `[i, :]` is the i th point.
 - `[:, 0:3]` is xyz.
 - `[:, 3:6]` is normalized xyz.
 - `[:, 6]` is the segmentation label.

i.e., each row is a point, and the columns are `[x, y, z, nx, ny, nz, label]`.

`--normal` flag will use all x-y-z-nx-ny-nz + label as input. Otherwise, only x-y-z + label will be used.

#### **OUTPUT**

- TRAINING: `<log_root>/part_seg/<args.log_dir or TIME>/checkpoints/best_model.pth`


### Define Segmentation Classes

In [7]:
# shapenet part segmentation
seg_classes = {
    'Earphone'  : [16, 17, 18],
    'Motorbike' : [30, 31, 32, 33, 34, 35],
    'Rocket'    : [41, 42, 43],
    'Car'       : [8, 9, 10, 11],
    'Laptop'    : [28, 29],
    'Cap'       : [6, 7],
    'Skateboard': [44, 45, 46],
    'Mug'       : [36, 37],
    'Guitar'    : [19, 20, 21],
    'Bag'       : [4, 5],
    'Lamp'      : [24, 25, 26, 27],
    'Table'     : [47, 48, 49],
    'Airplane'  : [0, 1, 2, 3],
    'Pistol'    : [38, 39, 40],
    'Chair'     : [12, 13, 14, 15],
    'Knife'     : [22, 23]
}

seg_ids = [seg_id for seg_val_sublist in seg_classes.values() for seg_id in seg_val_sublist]
len(seg_classes), len(seg_ids)

(16, 50)

### Training

`train_partseg.py` is used to train the model.

Check all the arguments:

In [8]:
!python3 train_partseg.py -h

usage: Model [-h] [--model MODEL] [--batch_size BATCH_SIZE] [--epoch EPOCH]
             [--learning_rate LEARNING_RATE] [--gpu GPU]
             [--optimizer OPTIMIZER] [--log_root LOG_ROOT] [--log_dir LOG_DIR]
             [--decay_rate DECAY_RATE] [--npoint NPOINT] [--normal]
             [--step_size STEP_SIZE] [--lr_decay LR_DECAY]
             [--data_dir DATA_DIR] [--notebook]

options:
  -h, --help            show this help message and exit
  --model MODEL         model name [default: pointnet_part_seg]
  --batch_size BATCH_SIZE
                        batch Size during training [default: 16]
  --epoch EPOCH         epoch to run [default: 251]
  --learning_rate LEARNING_RATE
                        initial learning rate [default: 0.001]
  --gpu GPU             specify GPU devices [default: 0]
  --optimizer OPTIMIZER
                        Adam or SGD [default: Adam]
  --log_root LOG_ROOT   Log root directory [default: log]
  --log_dir LOG_DIR     log path wihin log root direct

The following is the same as running:

```bash
python3 train_partseg.py \
     --model pointnet2_part_seg_msg \
     --normal \
     --log_dir pointnet2_part_seg_msg \
     # --log_root ../log \
     # --data_dir ../data/shapenetcore_partanno_segmentation_benchmark_v0_normal\
```

1635min on RTX4070super

In [ ]:
args = {
    'model'   : 'pointnet2_part_seg_msg',
    'normal'  : True, # in source: action='store_true'
    'log_dir' : 'pointnet2_part_seg_msg',
    'log_root': root_log_dir,
    'data_dir': partseg_data_dir,
    'notebook': True
}
partseg_train_args = train_partseg.CommandLineArgs(**args)
train_partseg.main(partseg_train_args, seg_classes)

### Testing

`test_partseg.py` is used to test the model.

Check all the arguments:

In [10]:
!python3 test_partseg.py -h

usage: PointNet [-h] [--batch_size BATCH_SIZE] [--gpu GPU]
                [--num_point NUM_POINT] [--log_root LOG_ROOT] --log_dir
                LOG_DIR [--normal] [--num_votes NUM_VOTES]
                [--data_dir DATA_DIR] [--notebook]

options:
  -h, --help            show this help message and exit
  --batch_size BATCH_SIZE
                        batch size in testing [default: 24]
  --gpu GPU             specify gpu device [default: 0]
  --num_point NUM_POINT
                        point Number [default: 2048]
  --log_root LOG_ROOT   Log directory root [default: log]
  --log_dir LOG_DIR     experiment root within log directory
  --normal              use normals
  --num_votes NUM_VOTES
                        aggregate segmentation scores with voting [default: 3]
  --data_dir DATA_DIR   data directory [default: data/shapenetcore_partanno_se
                        gmentation_benchmark_v0_normal]
  --notebook            set if running from jupyter notebook.


The following is the same as running:

```bash
python3 test_partseg.py \
     --normal \
     --log_dir pointnet2_part_seg_msg \
     # --log_root ../log \
     # --data_dir ../data/shapenetcore_partanno_segmentation_benchmark_v0_normal
```

In [11]:
args = {
    'normal'  : True, # in source: action='store_true'
    'log_dir' : 'pointnet2_part_seg_msg',
    'log_root': root_log_dir,
    'data_dir': partseg_data_dir,
    'notebook': True
}
partseg_test_args = test_partseg.CommandLineArgs(**args)
test_metrics, shape_ious, total_correct_class, total_seen_class = test_partseg.main(partseg_test_args, seg_classes)

PARAMETER ...
CommandLineArgs(normal=True, log_dir='pointnet2_part_seg_msg', log_root=PosixPath('/home/gokitune/Documents/pointnet-cleanup/log'), data_dir='/home/gokitune/Documents/pointnet-cleanup/data/shapenetcore_partanno_segmentation_benchmark_v0_normal', notebook=True, batch_size=24, gpu='0', num_point=2048, num_votes=3)
The number of test data is: 2874


  0%|          | 0/120 [00:00<?, ?it/s]

eval mIoU of Airplane       0.832425
eval mIoU of Bag            0.800677
eval mIoU of Cap            0.870462
eval mIoU of Car            0.795261
eval mIoU of Chair          0.909982
eval mIoU of Earphone       0.685080
eval mIoU of Guitar         0.912383
eval mIoU of Knife          0.864971
eval mIoU of Lamp           0.842101
eval mIoU of Laptop         0.958927
eval mIoU of Motorbike      0.741732
eval mIoU of Mug            0.949150
eval mIoU of Pistol         0.821926
eval mIoU of Rocket         0.617571
eval mIoU of Skateboard     0.769350
eval mIoU of Table          0.830951
Accuracy is: 0.94422
Class avg accuracy is: 0.87592
Class avg mIOU is: 0.82518
Instance avg mIOU is: 0.85636


In [12]:
test_metrics, shape_ious

({'accuracy': np.float64(0.9442248254827766),
  'class_avg_accuracy': np.float64(0.8759207130126322),
  'class_avg_iou': np.float64(0.8251842201256936),
  'instance_avg_iou': np.float64(0.8563595509095638)},
 {'Earphone': np.float64(0.6850799333063327),
  'Motorbike': np.float64(0.7417315267273756),
  'Rocket': np.float64(0.6175709218071446),
  'Car': np.float64(0.7952609539663411),
  'Laptop': np.float64(0.9589272268422386),
  'Cap': np.float64(0.8704616473313757),
  'Skateboard': np.float64(0.7693495385540174),
  'Mug': np.float64(0.9491496086794823),
  'Guitar': np.float64(0.912383098032691),
  'Bag': np.float64(0.8006769817962439),
  'Lamp': np.float64(0.8421009954636595),
  'Table': np.float64(0.8309506256034398),
  'Airplane': np.float64(0.8324252279445945),
  'Pistol': np.float64(0.8219257202527601),
  'Chair': np.float64(0.9099821483968256),
  'Knife': np.float64(0.8649713673065762)})

In [13]:
seg_correct = dict(zip(range(len(seg_ids)), total_correct_class))
seg_total = dict(zip(range(len(seg_ids)), total_seen_class))

seg_acc = {}
for id, correct_n in seg_correct.items():
    total_n = seg_total[id]
    if total_n == 0:
        seg_acc[id] = 0
    else:
        seg_acc[id] = correct_n / total_n
# print(seg_acc)

seg_class_acc = {}
for cat in seg_classes:
    seg_class_acc[cat] = {}
    for id in seg_classes[cat]:
        seg_class_acc[cat][id] = seg_acc[id]

pprint(seg_class_acc)

{'Airplane': {0: np.float64(0.9514596653832529),
              1: np.float64(0.9001144630170937),
              2: np.float64(0.843397780475117),
              3: np.float64(0.8994033412887829)},
 'Bag': {4: np.float64(0.6436188510491916), 5: np.float64(0.9850572482049291)},
 'Cap': {6: np.float64(0.9918728772440563), 7: np.float64(0.7586092715231788)},
 'Car': {8: np.float64(0.8897736050864579),
         9: np.float64(0.7818300939069666),
         10: np.float64(0.8562295569432055),
         11: np.float64(0.95768782709996)},
 'Chair': {12: np.float64(0.9623226967931763),
           13: np.float64(0.9608944573230654),
           14: np.float64(0.9315547931053914),
           15: np.float64(0.8571639237892998)},
 'Earphone': {16: np.float64(0.9554403101555813),
              17: np.float64(0.9524581307401405),
              18: np.float64(0.24307036247334754)},
 'Guitar': {19: np.float64(0.9526035152392864),
            20: np.float64(0.9122510283611172),
            21: np.float64(0.9

---


## Semantic Segmentation (S3DIS) 


### Data

- S3DIS dataset in text format
- Download `Stanford3dDataset_v1.2.zip` from:
  - [https://cvg-data.inf.ethz.ch/s3dis/](https://cvg-data.inf.ethz.ch/s3dis/)

Default `--data_dir` is `'data/s3dis/Stanford3dDataset_v1.2_Aligned_Version'`.

- `data/`
  - `s3dis/`
    - `ReadMe.txt`
    - `Stanford3dDataset_v1.2_Aligned_Version/`
      - `Area_1/`
        - `conferenceRoom_1/`
          - `Annotations/`
            - `beam_1.txt`
            - ...
          - `conferenceRoom_1.txt`
        - ...
      - ...

### **OUTPUT**

- TRAINING: `<log_root>/semantic_segmentation/<args.log_dir or TIME>/checkpoints/best_model.pth`
- PREDICTION: `None (only prints accuracy)`


### Data preparation

Use `data_utils/collect_indoor3d_data.py` to convert the dataset.

This will create a new directory `data/stanford_indoor3d/` (or whatever you specify with argument).

In [14]:
!python3 data_utils/collect_indoor3d_data.py -h

usage: collect_indoor3d_data.py [-h] [--data_path DATA_PATH]
                                [--output_path OUTPUT_PATH] [--skip_existing]

options:
  -h, --help            show this help message and exit
  --data_path DATA_PATH
                        Path to the indoor3d data directory. Default:
                        data/s3dis/Stanford3dDataset_v1.2_Aligned_Version
  --output_path OUTPUT_PATH
                        Path where processed data will be saved. Default:
                        data/stanford_indoor3d
  --skip_existing       If output file already exist, skip rather than
                        overwriting.


```bash
python3 data_utils/collect_inddoor3d_data.py \
    --data_path data/s3dis/Stanford3dDataset_v1.2_Alligned_Version \
    --output_path data/stanford_indoor3d \
    # --skip_existing
```

In [15]:
collect_indoor3d_data(
    data_path     = semseg_raw_data_dir,
    output_path   = semseg_data_dir,
    skip_existing = False,
)

/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version/Area_1/conferenceRoom_1/Annotations
/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version/Area_1/conferenceRoom_1/Annotations/door_1.txt
/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version/Area_1/conferenceRoom_1/Annotations/chair_10.txt
/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version/Area_1/conferenceRoom_1/Annotations/chair_3.txt
/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version/Area_1/conferenceRoom_1/Annotations/wall_4.txt
/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version/Area_1/conferenceRoom_1/Annotations/wall_3.txt
/home/gokitune/Documents/pointnet-cleanup/data/s3dis/Stanford3dDataset_v1.2_Aligned_Version/Area_1/conferenceRoom_1/Annotations/chair_7.txt
/home/gokitune/Documents/pointnet-

### Training

`train_semseg.py` is used to train the model.

Check all the arguments:

In [16]:
!python3 train_semseg.py -h

usage: Model [-h] [--model MODEL] [--batch_size BATCH_SIZE] [--epoch EPOCH]
             [--learning_rate LEARNING_RATE] [--gpu GPU]
             [--optimizer OPTIMIZER] [--log_root LOG_ROOT] [--log_dir LOG_DIR]
             [--decay_rate DECAY_RATE] [--npoint NPOINT]
             [--step_size STEP_SIZE] [--lr_decay LR_DECAY]
             [--test_area TEST_AREA] [--data_dir DATA_DIR] [--notebook]

options:
  -h, --help            show this help message and exit
  --model MODEL         model name [default: pointnet_sem_seg]
  --batch_size BATCH_SIZE
                        Batch Size during training [default: 16]
  --epoch EPOCH         Epoch to run [default: 32]
  --learning_rate LEARNING_RATE
                        Initial learning rate [default: 0.001]
  --gpu GPU             GPU to use [default: GPU 0]
  --optimizer OPTIMIZER
                        Adam or SGD [default: Adam]
  --log_root LOG_ROOT   Log root directory [default: log]
  --log_dir LOG_DIR     Log path [default: None]

The following is the same as running:

```bash
python3 train_semseg.py \
     --model pointnet2_sem_seg \
     --test_area 5 \
     --log_dir pointnet2_sem_seg
```

In [17]:
args = {
    'model'    : 'pointnet2_sem_seg',
    'test_area': 5,
    'log_dir'  : 'pointnet2_sem_seg',
    'log_root' : root_log_dir,
    'data_dir' : semseg_data_dir,
    'notebook' : True
}
semseg_train_args = train_semseg.CommandLineArgs(**args)
train_semseg.main(semseg_train_args)

PARAMETER ...
CommandLineArgs(model='pointnet2_sem_seg', test_area=5, log_dir='pointnet2_sem_seg', log_root=PosixPath('/home/gokitune/Documents/pointnet-cleanup/log'), data_dir='/home/gokitune/Documents/pointnet-cleanup/data/stanford_indoor3d', notebook=True, batch_size=16, epoch=32, learning_rate=0.001, gpu='0', optimizer='Adam', decay_rate=0.0001, npoint=4096, step_size=10, lr_decay=0.7)
start loading training data ...


  0%|          | 0/204 [00:00<?, ?it/s]

[1.124833  1.1816078 1.        2.2412012 2.340336  2.343587  1.7070498
 2.0335796 1.8852289 3.8252103 1.7948895 2.7857335 1.3452303]
Totally 47623 samples in train set.
start loading test data ...


  0%|          | 0/67 [00:00<?, ?it/s]

[1.1381457 1.2059734 1.        9.996554  2.5299199 2.0086675 2.1162353
 1.9657742 2.4815738 4.727607  1.4018297 2.8840992 1.4809785]
Totally 18923 samples in test set.
The number of training data is: 47623
The number of test data is: 18923
No existing model, starting training from scratch...
**** Epoch 1 (1/32) ****
Learning rate:0.001000
BN momentum updated to: 0.100000


...

eval mean loss: 1.024083
eval point avg class IoU: 0.506072
eval point accuracy: 0.820567
eval point avg class acc: 0.593221
------- IoU --------
class ceiling        weight: 0.091, IoU: 0.897 
class floor          weight: 0.200, IoU: 0.972 
class wall           weight: 0.165, IoU: 0.726 
class beam           weight: 0.278, IoU: 0.000 
class column         weight: 0.000, IoU: 0.039 
class window         weight: 0.018, IoU: 0.523 
class door           weight: 0.033, IoU: 0.062 
class table          weight: 0.030, IoU: 0.667 
class chair          weight: 0.040, IoU: 0.740 
class sofa           weight: 0.018, IoU: 0.369 
class bookcase       weight: 0.003, IoU: 0.598 
class board          weight: 0.112, IoU: 0.543 
class clutter        weight: 0.012, IoU: 0.444 

Eval mean loss: 1.024083
Eval accuracy: 0.820567
Best mIoU: 0.522748
**** Epoch 32 (32/32) ****
Learning rate:0.000343
BN momentum updated to: 0.012500


  0%|          | 0/2976 [00:00<?, ?it/s]

Training mean loss: 0.156998
Training accuracy: 0.945019
---- EPOCH 032 EVALUATION ----


  0%|          | 0/1182 [00:00<?, ?it/s]

eval mean loss: 0.927181
eval point avg class IoU: 0.511844
eval point accuracy: 0.820226
eval point avg class acc: 0.610392
------- IoU --------
class ceiling        weight: 0.091, IoU: 0.897 
class floor          weight: 0.200, IoU: 0.965 
class wall           weight: 0.166, IoU: 0.734 
class beam           weight: 0.277, IoU: 0.000 
class column         weight: 0.000, IoU: 0.062 
class window         weight: 0.019, IoU: 0.538 
class door           weight: 0.034, IoU: 0.187 
class table          weight: 0.031, IoU: 0.649 
class chair          weight: 0.039, IoU: 0.684 
class sofa           weight: 0.020, IoU: 0.349 
class bookcase       weight: 0.003, IoU: 0.629 
class board          weight: 0.109, IoU: 0.543 
class clutter        weight: 0.012, IoU: 0.417 

Eval mean loss: 0.927181
Eval accuracy: 0.820226
Best mIoU: 0.522748


### Testing

`test_semseg.py` is used to test the model.

Check all the arguments:

In [18]:
!python test_semseg.py -h

usage: Model [-h] [--batch_size BATCH_SIZE] [--gpu GPU]
             [--num_point NUM_POINT] [--log_root LOG_ROOT] --log_dir LOG_DIR
             [--visual] [--test_area TEST_AREA] [--num_votes NUM_VOTES]
             [--data_dir DATA_DIR] [--notebook]

options:
  -h, --help            show this help message and exit
  --batch_size BATCH_SIZE
                        batch size in testing [default: 32]
  --gpu GPU             specify gpu device
  --num_point NUM_POINT
                        point number [default: 4096]
  --log_root LOG_ROOT   Log directory root [default: log]
  --log_dir LOG_DIR     experiment root within log directory
  --visual              visualize result [default: False]
  --test_area TEST_AREA
                        area for testing, option: 1-6 [default: 5]
  --num_votes NUM_VOTES
                        aggregate segmentation scores with voting [default: 5]
  --data_dir DATA_DIR   data directory [default: data/s3dis/stanford_indoor3d]
  --notebook            s

The following is the same as running:

```bash
python3 test_semseg.py \
     --log_dir pointnet2_sem_seg \
     --test_area 5 \
     --visual \
     # --log_root ../log \
     # --data_dir ../data/stanford_indoor3d
```

In [19]:
args = {
    'log_dir'  : 'pointnet2_sem_seg',
    'test_area': 5,
    'visual'   : True, # in source: action='store_true'
    'log_root' : root_log_dir,
    'data_dir' : semseg_data_dir,
    'notebook' : True
}
semseg_test_args = test_semseg.CommandLineArgs(**args)
test_semseg.main(semseg_test_args)

PARAMETER ...
CommandLineArgs(log_dir='pointnet2_sem_seg', test_area=5, visual=True, log_root=PosixPath('/home/gokitune/Documents/pointnet-cleanup/log'), data_dir='/home/gokitune/Documents/pointnet-cleanup/data/stanford_indoor3d', notebook=True, batch_size=32, gpu='0', num_point=4096, num_votes=3)
The number of test data is: 67
---- EVALUATION WHOLE SCENE----
Inference [1/67] Area_5_conferenceRoom_2 ...


  0%|          | 0/3 [00:00<?, ?it/s]

[0.97585041 0.96757459 0.73715501 0.         0.         0.
 0.04169266 0.82866647 0.7936175  0.92112107 0.85284274 0.19090641
 0.63457004]
Mean IoU of Area_5_conferenceRoom_2: 0.6313
----------------------------
Inference [2/67] Area_5_office_20 ...


  0%|          | 0/3 [00:00<?, ?it/s]

[0.97339083 0.97720898 0.72474328 0.         0.         0.
 0.12621191 0.93313815 0.90328261 0.         0.49418326 0.
 0.34812771]
Mean IoU of Area_5_office_20: 0.6089
----------------------------
Inference [3/67] Area_5_office_26 ...


...

[0.97605425 0.98968205 0.70313472 0.         0.         0.
 0.35716293 0.         0.         0.         0.         0.
 0.61932751]
Mean IoU of Area_5_WC_2: 0.7291
----------------------------
Inference [66/67] Area_5_office_23 ...


  0%|          | 0/3 [00:00<?, ?it/s]

[0.9753747  0.97593794 0.65494632 0.         0.         0.51873042
 0.08184773 0.79825584 0.898943   0.         0.84459489 0.86953499
 0.57194372]
Mean IoU of Area_5_office_23: 0.6536
----------------------------
Inference [67/67] Area_5_office_19 ...


  0%|          | 0/3 [00:00<?, ?it/s]

[0.93527807 0.98600083 0.75450229 0.         0.         0.
 0.03609495 0.82574792 0.88002718 0.         0.53533335 0.6729121
 0.41316712]
Mean IoU of Area_5_office_19: 0.6710
----------------------------
------- IoU --------
class ceiling       , IoU: 0.887 
class floor         , IoU: 0.977 
class wall          , IoU: 0.738 
class beam          , IoU: 0.000 
class column        , IoU: 0.067 
class window        , IoU: 0.528 
class door          , IoU: 0.207 
class table         , IoU: 0.707 
class chair         , IoU: 0.759 
class sofa          , IoU: 0.442 
class bookcase      , IoU: 0.622 
class board         , IoU: 0.618 
class clutter       , IoU: 0.453 

eval point avg class IoU: 0.538855
eval whole scene point avg class acc: 0.624150
eval whole scene point accuracy: 0.830233
Done!
